In [2]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

import matplotlib
import IPython.display
import pickle
import os
from tqdm import tqdm
import collections
import json
import torch
import torch.nn.functional as F
import sys
import numpy as np


sys.path.append('..')
from src.train import Trainer
from src.utils import set_requires_grad
from clsmodel import mnist, stl10, afhq

import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

from src.comparative import LIME, SHAP, GradCAM, HyperbolicReasoning, DeepLIFT

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# load exp config
model_path = './LOGS/MNIST/models/best.pth'

#data-folder-path
folder_path = '../datasets/MNIST'

config = json.load(open(os.path.join(os.path.dirname(os.path.dirname(model_path)), 'exp-config.json'), 'r'))

# config['classifier'] = afhq(32, True).cuda(0)
config['classifier'] = mnist(32, 'it').cuda(0)

model = Trainer(**config)
model.load_model(model_path)
model.eval()


# model defns
feature_extractor = model.feature_extractor
conti_classifier = model.classifier_baseline
dis_classifier = model.classifier_quantized
decoder = model.dec
codebook_sampler = model.modelclass


# set_requires_grad(classifier, False)
set_requires_grad(feature_extractor, False)
set_requires_grad(conti_classifier, False)
set_requires_grad(dis_classifier, False)
set_requires_grad(decoder, False)
set_requires_grad(codebook_sampler, False)
    

# required config variables 
image_size = model.input_size
latent_dim = model.latent_size
num_classes = model.nclasses

In [ ]:
from src.dataset import get

batch_size = 16
train_loader, test_loader = get(batch_size, 
                      data_root = folder_path, 
                      train=True, val=True, 
                      input_size = image_size,
                      num_workers=10)


orig_images = []; orig_labels = []
for (img_batch, label_batch, *context_batch) in tqdm(test_loader):
    img_batch = img_batch.cuda(0)
    label_batch = label_batch.cuda(0)
    orig_images.extend(img_batch)
    orig_labels.extend(label_batch)


labels_ = np.array([lb.cpu().numpy() for lb in orig_labels])
# generate sample dataset
def get_sample_data(N_datapoints = 10): 
    sample_images = []; sample_labels = []
    for ci in range(num_classes):
        idxs = np.where(labels_ == ci)[0]
        randidxs = idxs[np.random.randint(0, len(idxs), N_datapoints//num_classes)]

        for idx in randidxs:
            sample_images.append(orig_images[idx].unsqueeze(0))
            sample_labels.append(orig_labels[idx].unsqueeze(0))

    sample_images = torch.cat(sample_images, dim = 0)
    sample_labels = torch.cat(sample_labels, dim = 0)
    return sample_images, sample_labels

In [ ]:
from src.inference import InductiveReasoningDT

inferer = InductiveReasoningDT(get_sample_data(100),
                               ncodebook_features = model.codebook_size,
                               nclasses = num_classes,
                               train_loader = train_loader,
                               feature_extractor = feature_extractor,
                               codebook = codebook_sampler,
                               classifier = dis_classifier,
                               decoder = decoder)

for ci in range(num_classes):
    inferer.get_class_tree(ci,f'./MNISTLOGS/class-{ci}')

In [ ]:
# init all explainers
import torch.nn as nn

class Classifier(nn.Module):
    def __init__(self, feature_extractor, classifier):
        super().__init__()
        self.feature_extractor = feature_extractor
        self.classifier = classifier
    
    def forward(self, x):
        x = self.feature_extractor(x)
        x = F.adaptive_avg_pool2d(x,(1,1)).view(x.size(0),-1)
        x = self.classifier(x)
        return x
    

classifier = Classifier(feature_extractor, classifier)
lime = LIME(classifier)
shap = SHAP(classifier)
deeplift = DeepLIFT(classifier)
gradcam = GradCAM(classifier, layeridx=-1)
hyperbolic = HyperbolicReasoning(inferer)

In [ ]:
sample_images, sample_labels = get_sample_data(10)

explainers = {'LIME': lime, 'SHAP':shap, 'deeplift': deeplift, 'gradcam': gradcam, 'hyperbolic' hyperbolic}

noise_weightage = [0.01, 0.1, 0.2, 0.3]
noise_type = ['gauss', 's&p', 'poisson', 'speckle']

for expname, exp in explainer.items():
    for nw in noise_weightage:
        for ny in noise_type:
            exp_robustness = exp.compute_robustness(sample_images, 
                                                    sample_labels,
                                                    type = ny, 
                                                    percentage = nw)
            print (f"Explainer Type: {expname}, Noise Type: {ny}, Noise Weightage: {nw}, Robustness: {exp_robustness}")